In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [5]:
import json
from datasets import Dataset


In [6]:
file_path = '/content/drive/MyDrive/training/train_data.json'


In [7]:

with open(file_path, 'r') as f:
    my_data = json.load(f)

In [8]:
# Check format 
formatted_data = {
    "instruction": [],
    "input": [],
    "output": []
}


In [9]:
for item in my_data:
    formatted_data["instruction"].append(item["instruction"])
    formatted_data["input"].append(item["input"])
    formatted_data["output"].append(item["output"])

In [10]:
dataset = Dataset.from_dict(formatted_data)

In [11]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


In [ ]:
dataset = Dataset.from_dict(formatted_data)
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:

#Inference
# run the model! change the instruction and input - leave the output blank
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Extract ESRS labels and ESG metrics from the input sentence and generate a JSON output with labels and values.", # instruction

        "Participants included roughly 50 staff plus international experts and start-ups focusing on ESG data analysis and decarbonisation technologies.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
import os
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import time
import re
import ast
import pandas as pd

# Define paths
input_dir = "/content/drive/MyDrive/SustainabilityReports/Firm_ID/text/71_80/BERT"
output_dir = "/content/drive/MyDrive/SustainabilityReports/Firm_ID/jsonfiles"
csv_path = "/content/drive/MyDrive/SustainabilityReports/Firm_ID/Results/71_80"
os.makedirs(output_dir, exist_ok=True)


max_length = 2048

def generate_json_output(line):
    instruction = "Extract ESRS labels and ESG metrics from the input sentence and generate a JSON output with labels and values."
    input_text = line  # Removed .strip()
    formatted_input = f"{instruction}\n\n{input_text}\n\n"

    # Tokenize the input text
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                instruction,  # instruction
                input_text,   # input
                ""            # output - leave this blank for generation!
            )
        ], return_tensors="pt", truncation=True, max_length=max_length).to("cuda")

    # Skip the line if the tokenized length exceeds max_length
    if inputs.input_ids.shape[-1] > max_length:
        print("Skipping line due to excessive length")
        return None

    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)


    generated_text = tokenizer.batch_decode(outputs)
    print("Response: ", generated_text)
    return " ".join(generated_text)

def list_to_string(lst):
    if lst is None:
        return ''
    return ', '.join(map(str, lst))

def create_csv(response_str):
    print(type(response_str))
    # Use regex to extract the JSON part
    json_match = re.search(r"### Response:\n({.*?})", response_str, re.DOTALL)
    if json_match:
        json_string = json_match.group(1)
        #  JSON issues
        json_string = json_string.replace("'", '"')  
        json_string = re.sub(r'\bNone\b', 'null', json_string)  
        try:
            #JSON string to Python dictionary
            response_dict = json.loads(json_string)
        except json.JSONDecodeError as e:
            raise ValueError(f"JSON decoding error: {e}")
    else:
        raise ValueError("Could not find the JSON part in the response string")

    #  input sentence
    input_match = re.search(r"### Input:\n(.*?)(?:\n\n|$)", response_str, re.DOTALL)

    if input_match:
        input_sentence = input_match.group(1).strip()
    else:
        raise ValueError("Could not find the input sentence in the response string")

    #  data for CSV
    data = {
        'Input': [input_sentence],
        'Category': list_to_string(response_dict.get('category')),
        'Indicator': list_to_string(response_dict.get('indicator')),
        'Topic': list_to_string(response_dict.get('topic')),
        'Trend': list_to_string(response_dict.get('trend')),
        'Units': list_to_string(response_dict.get('units')),
        'Value': list_to_string(response_dict.get('value')),
        'Year': list_to_string(response_dict.get('year'))
    }
    print(data)
    #  DataFrame
    df = pd.DataFrame(data)

    print("DF has been created successfully.")
    return df

# Process each file
for file_name in os.listdir(input_dir):
    if file_name.endswith(".txt"):
        input_file_path = os.path.join(input_dir, file_name)
        output_file_path = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}.json")

        combined_json_output = []
        dataframe_list = []
        with open(input_file_path, "r") as infile:
            for line in infile:
                print("Line is: ", line)
                if line.strip():  # Skip empty lines
                    json_output = generate_json_output(line)
                    print("JSON Completed")
                    combined_json_output.append(json_output)
                    try:
                        dataframe = create_csv(json_output)
                        dataframe_list.append(dataframe)
                    except ValueError as e:
                        print(f"Skipping line due to error: {e}")

        if dataframe_list:
            final_df = pd.concat(dataframe_list, ignore_index=True)
            csv_file_name = file_name[:-4] + '.csv'
            final_df.to_csv(os.path.join(csv_path, csv_file_name), index=False)
            print("CSV file has been created successfully.")
        else:
            print(f"No valid dataframes were created for {file_name}. Skipping CSV creation.")

        with open(output_file_path, "w") as outfile:
            json.dump(combined_json_output, outfile, indent=4)

print("Processing completed.")